In [10]:
# movie theatre simulation

In [1]:
import simpy as smp
import random
import statistics

In [13]:
wait_times = []

In [34]:
env = smp.Environment()

In [35]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        print('Done parking at %d' % env.now)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
        print('Done parking at %d' % env.now)

In [36]:
env.process(car(env))

<Process(car) object at 0x10351ad90>

In [37]:
env.run(until=20)

Start parking at 0
Done parking at 5
Start driving at 5
Done parking at 7
Start parking at 7
Done parking at 12
Start driving at 12
Done parking at 14
Start parking at 14
Done parking at 19
Start driving at 19


In [29]:
env.now

15

In [38]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [40]:
env2 = simpy.Environment()
car = Car(env2)
env2.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


# Interrupting a running car

In [64]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run1())

    def run1(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)


In [65]:
def driver(env, car):
    yield env.timeout(3)
    print('Going to interrupt')
    car.action.interrupt()

In [66]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Going to interrupt
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


# Shared resources

In [71]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req

        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [84]:
env3 = simpy.Environment()
bcs = simpy.Resource(env3, capacity=2) #creating two resources i-e charging station

In [85]:
for i in range(4):
    env3.process(car(env3, 'Car %d' % i, bcs, i*2, 5))

In [86]:
env3.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


# Basics

In [87]:
def example(env):
    value = yield env.timeout(1, value=42)
    print('now=%d, value=%d' % (env.now, value))

env = smp.Environment()
p = env.process(example(env))
env.run()

now=1, value=42


In [91]:
def example2(env):
    value = yield env.timeout(1, value=42)
    print('now=%d, value=%d' % (env.now, value))

In [90]:
env.process(example(env))
env.run()

now=3, value=42


In [92]:
env.process(example2(env))
env.run()

now=4, value=42


# Environments

In [107]:
env6 = smp.Environment()

In [108]:
for i in range(100):
    env6.run(until=i+8)
    print(env6.now)

8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


Instead of passing a number to run(), you can also pass any event to it. run() will then return when the event has been processed.

In [116]:
def my_proc(env):
    yield env.timeout(1)
    print(env.peek())
    yield env.timeout(10)
    return 'Monty pythons circus'

In [117]:
env = smp.Environment()
proc = env.process(my_proc(env))
env.run(until=proc)

inf


'Monty pythons circus'

In [118]:
env.now

11

In [114]:
until = 10
while env.peek() < until:
   env.step()

In [119]:
def subfunc(env):
    print(env.active_process)

def my_proc(env):
    while True:
        print(env.active_process)  # will print "p1"
        subfunc(env)
        yield env.timeout(1)

In [122]:
env = simpy.Environment()
p1 = env.process(my_proc(env))
print(env.active_process)
env.step()
print(env.active_process)

None
<Process(my_proc) object at 0x103d76510>
<Process(my_proc) object at 0x103d76510>
None


In [128]:
def my_callback(event):
    print('Called back from', event)

In [129]:
env = smp.Environment()
event = env.event()
event.callbacks.append(my_callback)
event.callbacks

[<function __main__.my_callback(event)>]

# Usages of events

In [131]:
class School:
    def __init__(self, env):
        self.env = env
        self.class_ends = env.event()
        self.pupil_procs = [env.process(self.pupil()) for i in range(3)]
        self.bell_proc = env.process(self.bell())

    def bell(self):
        for i in range(2):
            yield self.env.timeout(45)
            self.class_ends.succeed()
            self.class_ends = self.env.event()
            print()

    def pupil(self):
        for i in range(2):
            print(r' \o/', end='')
            yield self.class_ends

In [132]:
school = School(env)
env.run()

 \o/ \o/ \o/
 \o/ \o/ \o/


# Processes are events too

In [133]:
def sub(env):
    yield env.timeout(1)
    return 23

In [134]:
def parent(env):
    ret = yield env.process(sub(env))
    return ret

In [135]:
env.run(env.process(parent(env)))

23

## start a process after some delay

In [136]:
from simpy.util import start_delayed

def sub(env):
    yield env.timeout(1)
    return 23

def parent(env):
    sub_proc = yield start_delayed(env, sub(env), delay=3)
    ret = yield sub_proc
    return ret

In [137]:
env.run(env.process(parent(env)))

23

# Algo.

In [7]:
import simpy as smp
import random
import statistics
import numpy as np
import pandas as pd
import operator

In [8]:
#arr_rate_mean = [20,30,40]

# Simulating environment

In [9]:
def get_time_stamps(num_slices, quantum):
    time_stamps = random.sample(range(1, quantum), num_slices)
    return time_stamps

In [10]:
def get_value(mu, sigma):
    return np.random.normal(mu, sigma, 1)[0]

In [11]:
def generate_a_slice(class_type, time):
    a_slice = dict()
    # generate bid_value
    bid_value = np.floor(get_value(slice_req_params[class_type]['mean_bid_val'],
                          slice_req_params[class_type]['var']))
    
    # generate slice_duration
    slice_duration = np.floor(get_value(slice_req_params[class_type]['mean_duration'],
                          slice_req_params[class_type]['var']))

    # generate SLAV neglected
    SLAV_permit = slice_req_params[class_type]['SLAV']

    # generate cpu request
    ncpu = np.floor(get_value(slice_class_res_req[class_type]['NC'],
                     slice_req_params[class_type]['var']))

    # generate bw request
    nbw = np.floor(get_value(slice_class_res_req[class_type]['NB'],
                     slice_req_params[class_type]['var']))

    # generate memory request
    nm = np.floor(get_value(slice_class_res_req[class_type]['NM'],
                     slice_req_params[class_type]['var']))

    a_slice['bid_value']= bid_value
    a_slice['slice_duration']= slice_duration
    a_slice['SLAV_permit']= SLAV_permit
    a_slice['ncpu']= ncpu
    a_slice['nbw']= nbw
    a_slice['nm']= nm
    a_slice['class']= class_type
    a_slice['time']= time
    a_slice['end_time']= time + slice_duration

    return a_slice

In [12]:
def get_slices(class_type, quantum, curr_time):

    num_slices = int(max(1, np.floor(get_value(slice_req_params[class_type]['arr_rate_mean'], slice_req_params[class_type]['var']))))
    time_stamps = get_time_stamps(num_slices, quantum)
    time_stamps = list(map(lambda x: x + curr_time, time_stamps))
    time_stamps.sort()
    slices = [generate_a_slice(class_type, time_stamps[i]) for i in range(int(num_slices))]
    # print(len(time_stamps),' of ', class_type)
    return slices

# Decision algos.

In [13]:
existing_slices = list()

In [14]:
def release_slice(slice_, current_time):
    
    if slice_['end_time']<current_time:
        return 1
    else:
        return 0

In [15]:
def simple(slice_reqs):
    
    rej_penalty = 0
    accepted_slices = list()
    # assigning resources on fcfs basis
    for slice_ in slice_reqs:
        s_cpu = slice_['ncpu']
        s_m = slice_['nm']
        s_bw = slice_['nbw']
        
        if total_res['C'] >= s_cpu and total_res['M']>= s_m and total_res['B']>=s_bw:
            total_res['C'] -= s_cpu
            total_res['M'] -= s_m
            total_res['B'] -= s_bw
            accepted_slices.append(slice_)
        else:
            rej_penalty += slice_req_params[slice_['class']]['rej_penalty']
            continue

    return accepted_slices, rej_penalty

In [16]:
def release_slices(active_slices, curr_time):
    expired_slices_count = 0
    for slice_ in active_slices:
        if int(slice_['end_time'])<curr_time:
            total_res['C'] += slice_['ncpu']
            total_res['M'] += slice_['nm']
            total_res['B'] += slice_['nbw']
            active_slices.remove(slice_)
            expired_slices_count+=1
    return active_slices, expired_slices_count

In [17]:
def generate_slav_for_c1():
    num = np.random.exponential(3.45)
    if num>15:
        return 1

In [18]:
def generate_slav_for_c2():
    num = random.uniform(0, 1)
    if num > 0.9:
        return 1

In [19]:
def generate_slav_for_c3():
    num = random.uniform(0, 1)
    if num > 0.95:
        return 1

In [20]:
def calculate_slav(active_slices):
    slav = 0
    
    for slice_ in active_slices:
        if slice_['class']=='C1':
            val = generate_slav_for_c1()
            if val:
                slav+=slice_req_params['C1']['slav_penalty']
        elif slice_['class']=='C2':
            val = generate_slav_for_c2()
            if val:
                slav+=slice_req_params['C2']['slav_penalty']
        else:
            val = generate_slav_for_c3()
            if val:
                slav+=slice_req_params['C3']['slav_penalty']
                
    return slav

In [21]:
def calculate_revenue(active_slices):
    # calculate revenue
    active_revenue = 0
    for slice_ in active_slices:
        active_revenue += slice_['bid_value']
    return active_revenue

# Network window simulation

In [22]:
def a_net_win(env, NW, quantum):
    # get slices distributed till time t
    slice_list = list()
    active_slices = list()
    revenue = 0
    quantum_count = 0
    slav_penalty = 0
    
    while env.now < NW:
        
        # get slices which are well distributed in interval t
        new_slices = list()
        for class_ in class_types:
            new_slices.extend(get_slices(class_, quantum, env.now))
        
        
        new_slices.sort(key = operator.itemgetter('time'))
        # print(new_slices)
        # df = pd.DataFrame(new_slices)
        # df.to_excel('output.xlsx')
        
        # releasing resources
        active_slices, expired_slices_count = release_slices(active_slices, env.now)
        
        # select slices
        accepted_slices, rej_penalty = simple(new_slices)
        active_slices.extend(accepted_slices)
        
        # compute SLAV
        slav_penalty = calculate_slav(active_slices)
        
        # compute revenue
        curr_revenue = calculate_revenue(active_slices) - slav_penalty - rej_penalty
        revenue+= curr_revenue
        
        print(
                'Quantum no:', quantum_count,
                ', New slices generated:', len(new_slices), 
                ', Active slices:', len(active_slices), 
                ', Expired slices:', expired_slices_count,
                ', Current revenue:', curr_revenue,
                ', Rejection penalty:', rej_penalty,
                ', SLAV penalty:', slav_penalty
        )
        
        quantum_count +=1
        yield env.timeout(quantum)
        
    print('Total Revenue', revenue) 
        
    return slice_list

In [27]:
slice_req_params = {
    'C1':{'arr_rate_mean': 7, 'mean_bid_val': 25, 'var':1, 'mean_duration':90, 'SLAV':0, 'rej_penalty':100, 'slav_penalty':25},
    'C2':{'arr_rate_mean': 9, 'mean_bid_val': 15, 'var':1.5, 'mean_duration':70,'SLAV':5, 'rej_penalty':0, 'slav_penalty':15},
    'C3':{'arr_rate_mean': 12, 'mean_bid_val': 12, 'var':1.8, 'mean_duration':30,'SLAV':10, 'rej_penalty':0, 'slav_penalty':10}               
}

slice_class_res_req = {
    'C1':{'NC':4000, 'NB':1100, 'NM':2000},
    'C2':{'NC':2000, 'NB':500, 'NM':1500},
    'C3':{'NC':1500, 'NB':300, 'NM':1000}
}

total_res = {
    'C': 100000,
    'B': 70000,
    'M': 100000
}

class_types = ['C1', 'C2', 'C3']
quantum = 60
NW = 4000

In [28]:
def first_process(env):
    return_value = yield env.process(a_net_win(env, NW, quantum))
    df = pd.DataFrame.from_dict(return_value)
    df.to_excel("output.xlsx")

my_env = smp.Environment()
my_env.process(first_process(my_env))
my_env.run()

Quantum no: 0 , New slices generated: 23 , Active slices: 23 , Expired slices: 0 , Current revenue: 366.0 , Rejection penalty: 0 , SLAV penalty: 20
Quantum no: 1 , New slices generated: 28 , Active slices: 40 , Expired slices: 4 , Current revenue: 662.0 , Rejection penalty: 0 , SLAV penalty: 15
Quantum no: 2 , New slices generated: 29 , Active slices: 41 , Expired slices: 14 , Current revenue: 182.0 , Rejection penalty: 500 , SLAV penalty: 10
Quantum no: 3 , New slices generated: 28 , Active slices: 42 , Expired slices: 18 , Current revenue: 353.0 , Rejection penalty: 300 , SLAV penalty: 35
Quantum no: 4 , New slices generated: 24 , Active slices: 43 , Expired slices: 19 , Current revenue: 372.0 , Rejection penalty: 300 , SLAV penalty: 15
Quantum no: 5 , New slices generated: 27 , Active slices: 39 , Expired slices: 16 , Current revenue: 266.0 , Rejection penalty: 400 , SLAV penalty: 0
Quantum no: 6 , New slices generated: 28 , Active slices: 44 , Expired slices: 17 , Current revenue: 